In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.8 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Fri Feb 10 09:30:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    20W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import json
from abc import abstractmethod
from itertools import chain
import os
from typing import Dict, Iterable

drive_root = '/content/drive/MyDrive/Colab/vp'

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:

# with open(f"{drive_root}/arxiv-metadata-oai-snapshot-only-ml-categories.json", "r") as dataset_file:
#     df = pd.DataFrame.from_records((json.loads(line) for line in tqdm(dataset_file)))

In [ ]:
# df.to_pickle(f"{drive_root}/arxiv-metadata-oai-snapshot-only-ml-categories.pkl")
df = pd.read_pickle(f"{drive_root}/arxiv-metadata-oai-snapshot-only-ml-categories.pkl")

In [ ]:
from benchmarking import BaseSearchEngine, benchmark

In [ ]:
# import torch.nn.functional as F

# import torch
# from torch import Tensor
# from transformers import AutoTokenizer, AutoModel


# def average_pool(last_hidden_states: Tensor,
#                  attention_mask: Tensor) -> Tensor:
#     last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
#     return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# # Each input text should start with "query: " or "passage: ".
# # For tasks other than retrieval, you can simply use the "query: " prefix.
# input_texts = ['query: how much protein should a female eat',
#                'query: summit define',
#                "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#                "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]

# tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-base')
# model.to('cuda')
# model = AutoModel.from_pretrained('intfloat/e5-base')

# # Tokenize the input texts
# batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

# outputs = model(**batch_dict)
# embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# # (Optionally) normalize embeddings
# embeddings = F.normalize(embeddings, p=2, dim=1)
# print(embeddings.shape)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist())

In [ ]:
# batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to('cuda')


In [ ]:
import torch.nn.functional as F

import torch
from torch import Tensor, cat
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
print(device)

class E5SearchEngine(BaseSearchEngine):
    def __init__(self, dataset: pd.DataFrame):
        self.tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-base')
        self.model = AutoModel.from_pretrained('intfloat/e5-base').to(device)
        self.model.eval()
        self.mapping_to_doc_id = {i: _id for i, _id in enumerate(dataset['id'])}

    def average_pool(self, last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def embed(self, input_texts: list):
        embeddings = []

        # def makeshift_batch_splitter(input_texts: list, batch_size: int = 32):
        #     i = 0
        #     while i < len(input_texts):
        #         i+=batch_size
        #         yield input_texts[i-batch_size:i]

        with torch.no_grad():
            # for batch in tqdm(makeshift_batch_splitter(input_texts, batch_size=256), total=len(input_texts)/256):
            #     batch_dict = self.tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
            #     outputs = self.model(**batch_dict)
            #     embedding = self.average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
            #     embedding.to('cpu')
            #     embeddings.append(embedding)
            for text in tqdm(input_texts):
                batch_dict = self.tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
                outputs = self.model(**batch_dict)
                embedding = self.average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
                embedding.to('cpu')
                embeddings.append(embedding)
            return cat(embeddings)

    def index(self, dataset: pd.DataFrame, ids_column_name: str = 'id', abstract_column_name: str = 'abstract'):
        input_texts = [f"passage: {abstract}" for abstract in tqdm(dataset[abstract_column_name])]
        torch.save(self.embed(input_texts), f"{drive_root}/e5_index/embeddings_2nd.pt")

    def retrieve(self, query: str, amount: int) -> Iterable[str]:
        passage_embeddings = torch.load(f"{drive_root}/e5_index/embeddings.pt")
        query_embedded = self.embed([f"query: {query}"])
        return [self.mapping_to_doc_id[int(index)] for index in torch.topk(F.cosine_similarity(query_embedded, passage_embeddings), amount).indices.cpu()]

    def retrieve_scores(self, query: str, amount: int) -> Dict[str, float]:
        pass




cuda


In [ ]:
searcher = E5SearchEngine(df)

In [ ]:
%%time
searcher.retrieve("unet", 5)

100%|██████████| 1/1 [00:00<00:00, 76.12it/s]

CPU times: user 237 ms, sys: 809 ms, total: 1.05 s
Wall time: 1.92 s


['1912.05288', '1807.10165', '2010.13082', '2102.04738', '2012.00125']

In [ ]:
%%time
benchmark(df, searcher, f"{drive_root}/results/e5")

# Benchmarking

Tesla T4 (15GB)

### ColBert

And it is inconsistent sometimes it just does not end

Indexing:

10 abstracts: 1:44 - first run? after restart 

20: 0:35

100: 0:35

1000: 0:48

10000: 3:15

Total: 27:29
GPU RAM: 2.9 - 3.4gb
RAM: 4 - 6 gb

### E5

RAM used 5
GPU RAM 1.6gb-3gb (batch_size=1)
11.2-13.3gb(batch_size=256)

Total time:
CPU times: user 1h 16min 17s, sys: 31.2 s, total: 1h 16min 48s
Wall time: 1h 18min 26s

Retrieval:
57sec
1min:27s - second run wih more words